In [ ]:
# Google Colab Template:
# 1. Turn on GPU from Runtime -> Change runtime type -> GPU
# 2. Import Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 3. Copy the required content (datasets) from the drive (zip format)
!cp /content/drive/MyDrive/ML/ai-unibuc-23-31-2021.zip /content

# 4. Unzip the data from the copied .zip file for Colab local acces (faster dataset loading)
!unzip ai-unibuc-23-31-2021.zip

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as functional
import numpy as np
import cv2
import csv
from tqdm.notebook import trange, tqdm          # Status bar, docs: https://github.com/tqdm/tqdm#ipython-jupyter-integration
import os
import shutil

In [ ]:
# Fisierele de inputs
TRAIN_FOLDER = "./train"
TEST_FOLDER = "./test"
VALIDATION_FOLDER = "./validation"
TRAIN_LABELS = "./train.txt"
TEST_LABELS = "./test.txt"            # No labels
VALIDATION_LABELS = "./validation.txt"
RESHAPE_SIZE = 2500

In [ ]:
# Citirea imaginilor si datelor din fisiere
def read_images(path, files):
    if not os.path.isdir(path):
        raise Exception("Path-ul specificat nu este un folder")
    return torch.tensor([cv2.imread(f"{path}/{img}", 0).reshape(RESHAPE_SIZE) for img in files])

def read_csv_file(path):
    if not os.path.isfile(path):
        raise Exception("Path-ul specificat nu este un fisier")
    with open(path) as csv_file:
        data = np.array(list(csv.reader(csv_file)))
    return data

def split_csv_data(data):
    return data[:, 0], torch.tensor(data[:, 1].astype(int))

In [ ]:
# Pregateste structura pentru ImageFolder
def prepare_image_folder(folder, file):
    # Citim datele din CSV
    file_data = read_csv_file(file)
    
    if not os.path.isdir(folder):
        raise Exception("Path-ul specificat nu este un folder")
    imagefolder = folder + "-imagefolder"
    if os.path.isdir(imagefolder):
        os.rmdir(imagefolder)
    os.mkdir(imagefolder)
    
    for row in file_data:
        photo_name, label = row[0], row[1]
        if not os.path.isdir(f"{imagefolder}/{label}"):
            os.mkdir(f"{imagefolder}/{label}")
        shutil.copy(f"{folder}/{photo_name}", f"{imagefolder}/{label}/{photo_name}")
        
def prepare_both_set(folder1, file1, folder2, file2):
    # Citim datele din CSV
    file_data1 = read_csv_file(file1)
    file_data2 = read_csv_file(file2)

    if not os.path.isdir(folder1) or not os.path.isdir(folder2):
        raise Exception("Path-ul specificat nu este un folder")
    imagefolder = "both-imagefolder"
    if os.path.isdir(imagefolder):
        os.rmdir(imagefolder)
    os.mkdir(imagefolder)
    
    for row in file_data1:
        photo_name, label = row[0], row[1]
        if not os.path.isdir(f"{imagefolder}/{label}"):
            os.mkdir(f"{imagefolder}/{label}")
        shutil.copy(f"{folder1}/{photo_name}", f"{imagefolder}/{label}/{photo_name}")
    
    for row in file_data2:
        photo_name, label = row[0], row[1]
        if not os.path.isdir(f"{imagefolder}/{label}"):
            os.mkdir(f"{imagefolder}/{label}")
        shutil.copy(f"{folder2}/{photo_name}", f"{imagefolder}/{label}/{photo_name}") 

In [ ]:
# Creaza un Dataset din Train, Validation si Test
prepare_image_folder("train", "train.txt")
prepare_image_folder("validation", "validation.txt")
prepare_both_set("train", "train.txt", "validation", "validation.txt")

In [ ]:
# Making data
trainset = torchvision.datasets.ImageFolder(root="train-imagefolder", transform=train_data_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

validationset = torchvision.datasets.ImageFolder(root="validation-imagefolder", transform=validation_data_transform)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=32, shuffle=True)

bothset = torchvision.datasets.ImageFolder(root="both-imagefolder", transform=train_data_transform)
bothloader = torch.utils.data.DataLoader(bothset, batch_size=128, shuffle=True)

In [ ]:
# Transformations
train_data_transform = torchvision.transforms.Compose([
    # torchvision.transforms.Grayscale(num_output_channels=1),
    torchvision.transforms.Resize(370),
    torchvision.transforms.CenterCrop(350),
    torchvision.transforms.RandomRotation(5),
    torchvision.transforms.RandomHorizontalFlip(0.075),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    # torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

validation_data_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(370),
    torchvision.transforms.CenterCrop(350),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    # torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [ ]:
for img, label in trainloader:
    print(img.shape, label)
    break

torch.Size([32, 3, 230, 230]) tensor([0, 2, 1, 1, 2, 1, 2, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 2, 2,
        2, 2, 2, 1, 0, 1, 2, 2])


In [ ]:
class BasicConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super(BasicConvolutionalNeuralNetwork, self).__init__()
        self.ReLU = functional.relu
        self.Sig = functional.sigmoid
        self.ConvolutionLayer1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5)
        # 46 * 46 * 8
        self.ConvolutionLayer2 = nn.Conv2d(in_channels=8, out_channels=12, kernel_size=5)
        # 42 * 42 * 12
        self.ConvolutionLayer3 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3)
        # 40 * 40 * 20
        self.FullyConnectedLayer1 = nn.Linear(40 * 40 * 20, 1000)
        self.FullyConnectedLayer2 = nn.Linear(1000, 500)
        self.FullyConnectedLayer3 = nn.Linear(500, 50)
        self.FullyConnectedLayer4 = nn.Linear(50, 3)
    
    def forward(self, x):
        fx = self.ReLU(self.ConvolutionLayer1(x))
        # print(fx.shape)
        fx = self.ReLU(self.ConvolutionLayer2(fx))
        # print(fx.shape)
        fx = self.ReLU(self.ConvolutionLayer3(fx))
        # print(fx.shape)
        fx = fx.view(-1, 40 * 40 * 20)
        fx = self.ReLU(self.FullyConnectedLayer1(fx))
        fx = self.ReLU(self.FullyConnectedLayer2(fx))
        fx = self.ReLU(self.FullyConnectedLayer3(fx))
        fx = self.FullyConnectedLayer4(fx)
        return fx

In [ ]:
# Trainingul modelului
BCNN = BasicConvolutionalNeuralNetwork()

loss_function = nn.CrossEntropyLoss()

learning_rate = 1e-5
optimizer = torch.optim.Adam(BCNN.parameters(), lr=learning_rate)

for epoch in trange(3, desc="Epoch:"):
    rloss = 0.0
    for i, batch in enumerate(tqdm(trainloader)):
        x_batch, y_batch = batch
        optimizer.zero_grad()
        y_pred = BCNN(x_batch)
        loss = loss_function(y_pred, y_batch)
        optimizer.step()

        rloss += loss.item()
        if i % 20 == 19:
            print(f"[{epoch + 1}, {i + 1}] loss: {rloss / 2000}")
            rloss = 0.0
    print("Done training")

In [ ]:
accuracy = torch.tensor([])

with torch.no_grad():
    for data in validationloader:
        images, labels = data
        y_pred = BCNN(images)
        _, predicted = torch.max(y_pred, 1)
        # print(predicted)
        accuracy = torch.cat((accuracy, predicted == labels))
        print(np.mean(accuracy.numpy()))

In [ ]:
print(np.mean(accuracy.numpy()))

0.33333334


In [ ]:
class AlexNetPrototypeConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super(AlexNetPrototypeConvolutionalNeuralNetwork, self).__init__()
        self.ReLU = functional.relu
        self.Sig = functional.sigmoid
        self.ConvolutionLayer1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=3, padding=3)
        # 54 * 54 * 96
        self.ConvolutionLayer2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=3, padding=1)
        # 54 * 54 * 256
        self.MaxPool2 = nn.MaxPool2d(kernel_size=2)
        # 27 * 27 * 256
        self.ConvolutionLayer3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=11)
        # 17 * 17 * 384
        self.ConvolutionLayer4 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=1)
        # 17 * 17 * 256
        self.FullyConnectedLayer1 = nn.Linear(17 * 17 * 256, 4096)
        self.FullyConnectedLayer2 = nn.Linear(4096, 1024)
        self.FullyConnectedLayer3 = nn.Linear(1024, 100)
        self.FullyConnectedLayer4 = nn.Linear(100, 3)
    
    def forward(self, x):
        fx = self.ReLU(self.ConvolutionLayer1(x))
        # print(fx.shape)
        fx = self.ReLU(self.ConvolutionLayer2(fx))
        fx = self.MaxPool2(fx) 
        # print(fx.shape)
        fx = self.ReLU(self.ConvolutionLayer3(fx))
        # print(fx.shape)
        fx = self.ReLU(self.ConvolutionLayer4(fx))
        fx = fx.view(-1, 17 * 17 * 256)
        fx = self.ReLU(self.FullyConnectedLayer1(fx))
        fx = self.ReLU(self.FullyConnectedLayer2(fx))
        fx = self.ReLU(self.FullyConnectedLayer3(fx))
        fx = self.FullyConnectedLayer4(fx)
        return fx

In [ ]:
# Trainingul modelului
ANET = AlexNetPrototypeConvolutionalNeuralNetwork().cuda()

loss_function = nn.CrossEntropyLoss()

learning_rate = 1e-4
optimizer = torch.optim.Adam(ANET.parameters(), lr=learning_rate)

for epoch in trange(3, desc="Epoch:"):
    rloss = 0.0
    for i, batch in enumerate(tqdm(trainloader)):
        x_batch, y_batch = batch
        optimizer.zero_grad()
        y_pred = ANET(x_batch.cuda())
        loss = loss_function(y_pred.cuda(), y_batch.cuda())
        optimizer.step()

        rloss += loss.item()
        if i % 200 == 199:
            print(f"[{epoch + 1}, {i + 1}] loss: {rloss / 200}")
            rloss = 0.0
print("Done training")

In [ ]:
accuracy = torch.tensor([])

with torch.no_grad():
    for data in validationloader:
        images, labels = data
        y_pred = ANET(images.cuda())
        _, predicted = torch.max(y_pred, 1)
        # print(predicted)
        accuracy = torch.cat((accuracy, (predicted == labels.cuda()).cpu()))
        print(np.mean(accuracy.numpy()))

In [ ]:
# Another try
class AdvancedConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super(AdvancedConvolutionalNeuralNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=50, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=50, out_channels=24, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=24, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(9 * 9 * 16, 1000),
            nn.ReLU(),
            nn.Linear(1000, 100),
            nn.ReLU(),
            nn.Linear(100, 3)
        )
    
    def forward(self, x):
        return self.network(x)

In [ ]:
# Another try with AlexNet
# Good formula: [(W−K+2P)/S]+1, source: https://stackoverflow.com/questions/53580088/calculate-the-output-size-in-convolution-layer
class AlexNetOptimizedCNN(nn.Module):
    def __init__(self):
        super(AlexNetOptimizedCNN, self).__init__()
        self.ConvolutionLayers = nn.Sequential(
            nn.BatchNorm2d(3),
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=7, padding=3),
            nn.ReLU(),
            # 50 * 50 * 96
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=2, padding=1),
            nn.ReLU(),
            # 24 * 24 * 256
            nn.AdaptiveMaxPool2d((24, 24)),
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=12),
            nn.ReLU(),
            # 13 * 13 * 384
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=1),
            nn.ReLU(),
            # 13 * 13 * 384
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=1),
            nn.ReLU(),
            # 13 * 13 * 256
            nn.AdaptiveAvgPool2d((13, 13)),
            nn.Flatten(),
        )
        self.FullyConnectedLayers = nn.Sequential(
            nn.Linear(13 * 13 * 256, 4096),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.1),
            # nn.Linear(4096, 1000),
            # nn.ReLU(),
            # nn.Dropout(),
            # nn.Linear(1000, 200),
            # nn.ReLU(),
            nn.Linear(4096, 3)
        ) 
    
    def forward(self, x):
        fx = self.ConvolutionLayers(x)
        fx = self.FullyConnectedLayers(fx)
        return fx

In [ ]:
# Trying with VGG Convolutional Neural Network
# Good formula: [(W−K+2P)/S]+1, source: https://stackoverflow.com/questions/53580088/calculate-the-output-size-in-convolution-layer
class VGG_CNN(nn.Module):
    def __init__(self):
        super(VGG_CNN, self).__init__()
        self.ConvolutionLayers = nn.Sequential(
            # Trying the E version of VGG
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            # (50 - 2) / 2 + 1 = 25

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            # (25 - 2) / 2 + 1 = 12

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            # nn.AdaptiveMaxPool2d((10, 10)),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            # nn.AdaptiveMaxPool2d((9, 9)),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
        )
        self.AveragePool = nn.AdaptiveAvgPool2d((7, 7))
        self.FullyConnectedLayers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7 * 7 * 512, 4096),
            nn.ReLU(True),
            nn.Dropout(0.05),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(0.05),
            nn.Linear(4096, 3),
        )
    
    def forward(self, x):
        fx = self.ConvolutionLayers(x)
        fx = self.AveragePool(fx)
        fx = self.FullyConnectedLayers(fx)
        return fx

In [ ]:
# Trying with VGG Convolutional Neural Network (B variant (without skip connection))
# Good formula: [(W−K+2P)/S]+1, source: https://stackoverflow.com/questions/53580088/calculate-the-output-size-in-convolution-layer
class VGG_Simple(nn.Module):
    def __init__(self):
        super(VGG_Simple, self).__init__()
        self.ConvolutionLayers = nn.Sequential(
            # Trying the B version of VGG
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            # (50 - 2) / 2 + 1 = 25

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            # (25 - 2) / 2 + 1 = 12

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            # nn.AdaptiveMaxPool2d((10, 10)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # (12 - 2) / 2 + 1 = 6

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            # nn.AdaptiveMaxPool2d((5, 5)),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
        )
        self.AveragePool = nn.AdaptiveAvgPool2d((7, 7))
        self.FullyConnectedLayers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7 * 7 * 512, 4096),
            nn.ReLU(True),
            nn.Dropout(0.05),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(0.05),
            nn.Linear(4096, 3),
        )
    
    def forward(self, x):
        fx = self.ConvolutionLayers(x)
        fx = self.AveragePool(fx)
        fx = self.FullyConnectedLayers(fx)
        return fx

In [ ]:
# Definirea datelor
# advanced_network = AdvancedConvolutionalNeuralNetwork().cuda()
# advanced_network = VGG_Simple().cuda()
# advanced_network = VGG_CNN().cuda()
loss_function = nn.CrossEntropyLoss().cuda()

In [ ]:
# Optimizer
learning_rate = 1e-5
optimizer_function = torch.optim.Adam(advanced_network.parameters(), lr=learning_rate)
# optimizer_function = torch.optim.Adadelta(advanced_network.parameters(), lr=learning_rate)
# optimizer_function = torch.optim.Adagrad(advanced_network.parameters(), lr=learning_rate)
# optimizer_function = torch.optim.SGD(advanced_network.parameters(), lr=learning_rate, nesterov=True, momentum=0.9)

In [ ]:
def train_model(data, epochs):
    loss_rate = 0.0
    correct_labels = 0
    total_data = 0
    total_batches = len(data)
    for epoch in trange(epochs, desc="Epoch:"):
        advanced_network.train()
        for photos, labels in tqdm(data):
            photos = photos.cuda()
            labels = labels.cuda()
            y_predictions = advanced_network(photos)
            correct_labels += torch.sum(y_predictions.argmax(dim=1) == labels)
            total_data += len(photos)
            loss = loss_function(y_predictions, labels)
            loss_rate += loss.item()

            # Learning
            optimizer_function.zero_grad()
            loss.backward()
            optimizer_function.step()
        print(f"Epoch {epoch}, Loss = {loss_rate / total_batches}, Accuracy = {correct_labels / total_data}")
        loss_rate = 0.0
        correct_labels = 0
        total_data = 0
    
        # Testing on validation
        print(f"Validation after epoch {epoch}: ")
        evaluate_model(validationloader)

        # Making submissions
        test_labels = compute_predictions(testloader).cpu().numpy()
        write_to_file(epoch, test_labels)

In [ ]:
train_model(trainloader, 30)

RuntimeError: ignored

In [ ]:
# Functie pentru evaluarea modelului pe validation data
def evaluate_model(data):
    advanced_network.eval()
    accuracy = 0
    total = 0
    loss_rate = 0.0
    with torch.no_grad():
        for photos, labels in tqdm(data):
            photos = photos.cuda()
            labels = labels.cuda()
            y_predictions = advanced_network(photos)
            loss = loss_function(y_predictions, labels)
            loss_rate += loss.item()
            accuracy += torch.sum(y_predictions.argmax(dim=1) == labels)
            total += len(photos)
        print(f"Loss = {loss_rate / len(data)}, Accuracy = {accuracy / total}")

In [ ]:
evaluate_model(validationloader)


Accuracy = 0.3382222354412079


In [ ]:
# Functie care face predictiile pentru test data
def compute_predictions(data):
    advanced_network.eval()
    labels = torch.tensor([]).cuda()
    with torch.no_grad():
        for photos, fake_labels in tqdm(data):
            photos = photos.cuda()
            y_predictions = advanced_network(photos)
            labels = torch.cat((labels, torch.argmax(y_predictions, dim=1)))
        return labels

In [ ]:
!mkdir test-imagefolder
!cp -r test test-imagefolder/test

In [ ]:
# Loading the test data
testset = torchvision.datasets.ImageFolder(root="test-imagefolder", transform=validation_data_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 32, shuffle = False)

In [ ]:
test_labels = compute_predictions(testloader).cpu().numpy()
print(test_labels)

KeyboardInterrupt: ignored

In [ ]:
# Write to submission file
!mkdir submissions_folder

def write_to_file(epoch, test_labels):
    with open(f"submissions_folder/submission_epoch_{epoch}.csv", "w") as csvfile:
        csvfile.write("id,label\n")
        for i in range(len(test_labels)):
            file_name = testloader.dataset.samples[i][0].split("/")[-1]
            label = int(test_labels[i])
            # print(file_name, label)
            csvfile.write(f"{file_name},{label}\n")

In [ ]:
# Saving the model
torch.save(advanced_network.state_dict(), "models")

In [ ]:
# ResNet with Transfer Learning
advanced_network = torchvision.models.resnet101(pretrained=True).cuda()
last_layer = advanced_network.fc.in_features
advanced_network.fc = nn.Linear(in_features=last_layer, out_features=3).cuda()

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


In [ ]:
# Inception
advanced_network = torchvision.models.inception_v3(pretrained=True).cuda()
last_layer = advanced_network.fc.in_features
advanced_network.fc = nn.Linear(in_features=last_layer, out_features=3).cuda()

In [ ]:
# DenseNet
advanced_network = torchvision.models.densenet201(pretrained=False).cuda()
last_layer = advanced_network.classifier.in_features
advanced_network.classifier = nn.Linear(in_features=last_layer, out_features=3).cuda()

In [ ]:
advanced_network

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel